In [16]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from topogram_client import TopogramAPIClient

from csv import DictReader
import os
import json
import pymongo
#import musicbrainzngs
import arrow
from datetime import datetime
from pymongo import MongoClient
from collections import defaultdict
from dateutil import parser
from datetime import timedelta
import pickle
import time
from slugify import slugify
from fuzzywuzzy import fuzz
import urllib
import math

##SET PATH TO SAVE FILES:
savpath="./bands_dicts/"


In [17]:
#a function to have a range of colors for nodes/edges 
import colorsys
def get_N_HexCol(N=20):

    HSV_tuples = [(x*1.0/N, 0.5, 0.5) for x in xrange(N)]
    hex_out = []
    for rgb in HSV_tuples:
        rgb = map(lambda x: int(x*255),colorsys.hsv_to_rgb(*rgb))
        hex_out.append("".join(map(lambda x: chr(x).encode('hex'),rgb)))
    return hex_out

In [18]:
def create_topogram(title, nodes, edges):

    print "Creating topogram '%s'"%title

    try :
        r = topogram.create_topogram(title)
    except ValueError:
        print '> Topogram already exists'
        r = topogram.get_topogram_by_name(title)
    print r
    topogram_ID = r["data"]["_id"]
    print "topogram ID : %s"%topogram_ID

    # get and backup existing nodes and edges
    existing_nodes = topogram.get_nodes(topogram_ID)["data"]
    existing_edges = topogram.get_edges(topogram_ID)["data"]

    # clear existing graph
    if len(existing_nodes):
        topogram.delete_nodes([n["_id"] for n in existing_nodes])
        print "%s nodes deleted"%len(existing_nodes)
    if len(existing_edges):
        topogram.delete_edges([n["_id"] for n in existing_edges])
        print "%s edges deleted"%len(existing_edges)

    r = topogram.create_nodes(topogram_ID, nodes)
    #print r
    print "%s nodes created."%len(r["data"])
    r = topogram.create_edges(topogram_ID, edges)
    print "%s edges created."%len(r["data"])

    print "done. Topogram has been updated. Check it at %s/topograms/%s"%(TOPOGRAM_URL, topogram_ID)



In [14]:
artist={}
artist["name"]="deerhoof"

with open("../../4_miner/tour_miner/"+slugify(artist["name"]+artist["mbid"])+".obj", 'rb') as handle:
          tempL= pickle.load( handle)
print tempL

{1: {'tourdist': 2951.729403951149, 'tourdistOpt': 3430.7929093463886}, 2: {'tourdist': 1722.7256763733676, 'tourdistOpt': 1722.7256763733676}, 3: {'tourdist': 10603.784273888785, 'tourdistOpt': 10522.516818819335}, 4: {'tourdist': 1778.9487958304094, 'tourdistOpt': 2478.939515602156}, 5: {'tourdist': 10803.835620111713, 'tourdistOpt': 10065.397578887305}, 6: {'tourdist': 1752.375554214177, 'tourdistOpt': 1888.8080221699468}, 7: {'tourdist': 11292.690515977998, 'tourdistOpt': 12571.303667491313}, 8: {'tourdist': 885.322283400078, 'tourdistOpt': 885.322283400078}, 9: {'tourdist': 15453.752812126802, 'tourdistOpt': 21442.560183601046}, 10: {'tourdist': 16297.238927266995, 'tourdistOpt': 10577.63962813819}, 11: {'tourdist': 4295.480813158602, 'tourdistOpt': 7963.9834223382295}, 12: {'tourdist': 21537.033492529532, 'tourdistOpt': 21150.55457763848}, 13: {'tourdist': 3962.5974404832386, 'tourdistOpt': 3847.9781411136037}, 14: {'tourdist': 2079.1038413122005, 'tourdistOpt': 1934.980546102542

In [21]:
# setup mongo
client = MongoClient()
db = client["bandstour"]
bandsintown =  db["minedArtists"]

# credentials
TOPOGRAM_URL = "https://app.topogram.io"
USER = "gregory.bahde@laposte.net"
PASSWORD = "matrix"

# data
#title = "Foo Fighters bandstour 0.1"
my_nodesdict = {}
my_nodes= []
my_edges = []
my_edgesdict = {}
my_nodesdictGLOBAL={}

#LIVENATION CATALOGUE
liste_salles_LN = json.load(open('./JSON_LIVENATION_VENUE.json'))

liste_salle_parsee_LN={}
for salle in liste_salles_LN:
    if liste_salles_LN[salle]["type"]=="venue":
 
            liste_salle_parsee_LN[slugify(liste_salles_LN[salle]["title"])]={"lat":liste_salles_LN[salle]["position"]["lat"],"lng":liste_salles_LN[salle]["position"]["lng"]}
#print liste_salle_parsee_LN


# date parsing
DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
DATETIME_FORMAT_FOR_MSBNZ ='%Y-%m-%d'

#retrieve trip coords for each artist
dict_coords={}
colors_for_nodes_and_edges = get_N_HexCol()

for artist in db.minedArtists.find({"totalKm": {"$gte": 300000}}) :
 title = artist["name"]   
 if os.path.exists(savpath+slugify(artist["name"]+str(artist["mbid"]))+".obj")==True:
        my_nodesdict = {}
        my_nodesdictList=[]
        print "SKIPPED PROCESSING BUT TAKING INTO ACCOUNT ",artist["name"]
        with open(savpath+slugify(artist["name"]+str(artist["mbid"]))+".obj", 'rb') as handle:
          tempL= pickle.load( handle)
    #print 
        tit=tempL[0].encode("utf-8")
        #edgestop=temp[2]
        my_nodesdictList=tempL[1]
        for node in my_nodesdictList:
            my_nodesdict[node["data"]["id"]]=node

        #print my_nodes
        #pass
 else:
    my_nodesdict = {}
    my_nodes= []
    my_edges = []
    my_nodesdictList=[]
    my_edgesdict = {}
    #colors_for_nodes_and_edges = get_N_HexCol()
    
    print artist["name"]
    #print artist
    dates_coords = []
    dates = artist.get("gigs")
    tours = artist.get("tours")    
    singlegigs = artist.get("singleGigs")
    dateini=0
    touring=0
    tour_indice=0
    tour_counter_for_CO2=1
    tour_counter_for_CO2_lock=0
    tmpOLDgig ={}
    with open("../../4_miner/tour_miner/"+slugify(artist["name"])+".obj", 'rb') as handle:
                    tempCO2= pickle.load( handle)
    #print tempCO2
    #print tours
    for tour in tours:
        
        ##WE NEED TO MATCH TOUR INDICES OF CO2 SPECS FILES...
     if tour_counter_for_CO2_lock == 1:
        tour_counter_for_CO2+=1
     tour_counter_for_CO2_lock=0
     tour_color = "#"+colors_for_nodes_and_edges[tour_indice % 20]
     #print tour_color
     #print tour
     gig_indice =0
     dates = tour.get("gigs")
     for date in dates:
        #print date.get("datetime")
        venue = date.get("venue")
        latitude = venue.get("latitude")
        longitude = venue.get("longitude")
        #print venue
        ###concat key
        idd= slugify(venue["name"])+str(int(venue["latitude"]))+str(int(venue["longitude"]))
        
        if idd not in my_nodesdict:
            
            node = {
                    "id" : idd,
                    "name" : slugify(venue["name"]),
                    "lat" : float(venue["latitude"]),
                    "lng" : float(venue["longitude"]),
                    "weight" : float(1),
                    "start" : date.get("datetime").isoformat(),
                    "end" : (date.get("datetime")+timedelta(days=1)).isoformat(),
                    "color" : tour_color,
                    "notes" : "name : "+slugify(venue["name"])+"  \nlat : "+str(float(venue["latitude"]))+"  \nlng :"+str(float(venue["longitude"]))+"  \n weight :"+str(1)+"  \nstart : "+str(date.get("datetime").isoformat())+"  \nend : "+str((date.get("datetime")+timedelta(days=1)).isoformat())+"  \ncolor :"+str(tour_color)+"  \n  <a href=\"https://google.com/search?q="+slugify(venue["name"])+"\" target=\"_blank\">"+slugify(venue["name"])+"</a>"
                #+"  \n[search venue named "+slugify(venue["name"])+" on google](https://google.com/search?q="+slugify(venue["name"])+")"
                    }
        else:
                
                node = {
                    "id" : idd,
                    "name" : slugify(venue["name"]),
                    "lat" : float(venue["latitude"]),
                    "lng" : float(venue["longitude"]),
                    "weight" : math.sqrt(float(my_nodesdict[idd]["data"].get("weight"))+1) ,
                    "start" : my_nodesdict[idd]["data"].get("start"),
                    "end" : (date.get("datetime")+timedelta(days=1)).isoformat(),
                    "color" : tour_color,
                    "notes" : "name : "+slugify(venue["name"])+"  \nlat : "+str(float(venue["latitude"]))+"  \nlng :"+str(float(venue["longitude"]))+"  \n weight :"+str((float(my_nodesdict[idd]["data"].get("weight"))+1)**2)+"  \nstart : "+str(my_nodesdict[idd]["data"].get("start"))+"  \nend : "+(date.get("datetime")+timedelta(days=1)).isoformat()+"  \ncolor :"+tour_color+"  \n  <a href=\"https://google.com/search?q="+slugify(venue["name"])+"\" target=\"_blank\">"+slugify(venue["name"])+"</a>"
                    #+"  \n[search venue named "+slugify(venue["name"])+" on google](https://google.com/search?q="+slugify(venue["name"])+")"
                    
                    }
        if dateini == 0:
                dateini =1
        else:
                #print my_nodes[-1]["data"]["id"]
                #print dates[gig_indice-1]["distanceToNextGig"]
                
                
                
                notesE=""
                notesE+="distance : "
                try: 
                    notesE+=str(dates[gig_indice]["distanceToNextGig"])
                except KeyError:
                    notesE+="dates[gig_indice][distanceToNextGig] NOT AVAILABLE"
                notesE+=" km  \n"
                notesE+="source : "
                notesE+=str(my_nodes[-1]["data"]["id"])
                notesE+="  target : "
                notesE+=str(node["id"])
                notesE+="  \ndatesource : "
                notesE+=dates[gig_indice -1]["datetime"].isoformat()
                notesE+="  \ndatetarget : "
                notesE+=dates[gig_indice]["datetime"].isoformat()
                notesE+="  \ngroup: "
                notesE+=str(int(tour_indice)+1)
                #notesE+=
                
                
                #print gig_indice
                edge = {
                    "source" : my_nodes[-1]["data"]["id"],
                    "target" : node["id"],
                    "color" :  tour_color,
                    "notes" : notesE,
                    #"weight" : float(e["weight"]),
                    "start" : dates[gig_indice -1]["datetime"].isoformat(),
                    "end" : dates[gig_indice]["datetime"].isoformat(),
                    "group" : str(tour_counter_for_CO2)
                    
                }
                #print edge
                #ADDING EDGES GROUPS/CO2 HERE
                
                if tour["nbGigs"] > 2:
                    try:
                        if tempCO2[tour_counter_for_CO2]['tourdist'] > tempCO2[tour_counter_for_CO2]['tourdistOpt']:
                            edge["group"]+=" DASHED"
                            edge["notes"]+="  \n OPTIMISABLE  \nDistance parcourue pendant le tour: "+str(tempCO2[tour_counter_for_CO2]['tourdist'])+" km  \n"
                            edge["notes"]+="Distance recalculée: "+str(tempCO2[tour_counter_for_CO2]['tourdistOpt'])+" km  \n"
                            try:
                                edge["notes"]+="Taux d'optimisation "+str((tempCO2[tour_counter_for_CO2]['tourdist']-tempCO2[tour_counter_for_CO2]['tourdistOpt'])/tempCO2[tour_counter_for_CO2]['tourdist'])+" %"
                            except:
                                print "OUPS"
                        else:

                            edge["notes"]+="  \nDistance parcourue pendant le tour: "+str(tempCO2[tour_counter_for_CO2]['tourdist'])+" km  \n"
                        tour_counter_for_CO2_lock=1
                    except KeyError as e:
                        print e
                else:
                    #print "TOUR NON COMPTE ", tour_indice, "//",tour_counter_for_CO2
                    continue
                    
                    
                    
                my_edges.append({ "data" : edge })
        my_nodes.append({ "data" : node })   
        my_nodesdict[node["id"]]= { "data" : node }
        gig_indice+=1
    #print my_nodes
    #print "<=============================================>"
    #print my_nodesdict
    #print "<=============================================>"
    #print my_edges
     tour_indice +=1
        #print "coords",latitude,"/",longitude
    for key, value in my_nodesdict.iteritems():
            my_nodesdictList.append(value)
    
    #ADDING LIVENATION DETECTOR

    for node in my_nodesdictList:
        for salle in liste_salle_parsee_LN:
            if fuzz.partial_ratio( node['data']['name'], salle)>=80:
                #print "ONE HEERE"
                if round(node['data']['lat'],2)==round(liste_salle_parsee_LN[salle]["lat"],2) and round(node['data']['lng'],2)==round(liste_salle_parsee_LN[salle]["lng"],2) :
                    #print "CONFIRMED LIVENATION!"
                    node['data']['color']="#ff0000"
                    node['data']["notes"]+="  \nLivenation venue"
        #if liste_salle_parsee_LN
        
        
    #ADDING EDGES WEIGHT
    terp= map(lambda x : str(sorted([x["data"]["source"],x["data"]["target"]])),my_edges)
    
    #print terp
    from collections import Counter
    c= Counter()
    for ter in terp:
        c[ter]+=1
    #print c
    for edge in my_edges:
        if str(sorted([edge["data"]["source"],edge["data"]["target"]])) not in terp:
            print "BUG!!!!"
        else:
            if edge["data"]["source"] == edge["data"]["target"]:
                #print c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))]
                edge["data"]["notes"]+="  \ncounted "+str(c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))])+"  times"
                edge["data"]["weight"]= float(1)
                #print "a"
            else:
                edge["data"]["notes"]+="  \ncounted "+str(c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))])+"  times"
                edge["data"]["weight"]= float(c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))])
                #print "b"
    #print my_edges
    
    
    
    with open(savpath+slugify(artist["name"]+str(artist["mbid"]))+".obj", 'wb') as handle:
                            pickle.dump([artist["name"]+"/BETA_0.3",my_nodesdictList,my_edges], handle, protocol=pickle.HIGHEST_PROTOCOL)    
 for key, value in my_nodesdict.iteritems():
            #print "key",key,"/////",value
            if key in my_nodesdictGLOBAL:
                #print "TROUVE"
                #print "key",key,"/////",value
                #print "GLOBAL",my_nodesdictGLOBAL[key]
                my_nodesdictGLOBAL[key]['data']["weight"]=math.sqrt(my_nodesdictGLOBAL[key]['data']["weight"]**2+my_nodesdict[key]['data']["weight"]**2)
                if time.strptime(my_nodesdictGLOBAL[key]['data']["start"],DATETIME_FORMAT)>time.strptime(my_nodesdict[key]['data']["start"],DATETIME_FORMAT):
                    my_nodesdictGLOBAL[key]['data']["start"]=my_nodesdict[key]['data']["start"]
                if time.strptime(my_nodesdictGLOBAL[key]['data']["end"],DATETIME_FORMAT)<time.strptime(my_nodesdict[key]['data']["end"],DATETIME_FORMAT):
                    my_nodesdictGLOBAL[key]['data']["end"]=my_nodesdict[key]['data']["end"]
                my_nodesdictGLOBAL[key]['data']["notes"]+="  \n"+slugify(artist["name"])
                my_nodesdictGLOBAL[key]['data']["count"]+=int(1)
                #print "GLOBAL2",my_nodesdictGLOBAL[key]
            else:
                my_nodesdictGLOBAL[key]=value
                #print my_nodesdictGLOBAL[key]
                my_nodesdictGLOBAL[key]['data']["notes"]="  \n List of the bands that played in this venue:  \n"+slugify(artist["name"])
                my_nodesdictGLOBAL[key]['data']["count"]=int(1)
for key in my_nodesdictGLOBAL:
    my_nodesdictGLOBAL[key]['data']["notes"]+="  \n TOTAL: "+str(my_nodesdictGLOBAL[key]['data']["count"])+" Bands"
    
    
with open("./my_nodesdictGLOBAL.obj", 'wb') as handle:
                            pickle.dump(my_nodesdictGLOBAL, handle, protocol=pickle.HIGHEST_PROTOCOL)

SKIPPED PROCESSING BUT TAKING INTO ACCOUNT  Israel Nash
SKIPPED PROCESSING BUT TAKING INTO ACCOUNT  Gilberto Santa Rosa
SKIPPED PROCESSING BUT TAKING INTO ACCOUNT  Delhi 2 Dublin
Iron Reagan
The Ben Miller Band
Miley Cyrus
Space Jesus
Tiger Army
Fantasia
Con Brio
Turnover
Martyn Joseph
BOO SEEKA
Sodom
Jackyl
Taylor McFerrin
Mia
Cheech
Peelander-Z
Idina Menzel
Cherish the Ladies
John McLaughlin
Sebastian
Steely Dan
Sawyer Brown
Mimi
Pedrito Martinez Group
Joey Anderson
George Winston
Joe Lynn Turner
Fujiya & Miyagi
Propagandhi
Kashmir
Blind Guardian
Bleached
Roberto Fonseca
Insomnium
The Sonics
Gregory Alan Isakov
Pato Banton
Tory Lanez
Hannibal Buress
Gabriel Kelley
Pelican
Nifra
Theo Croker
Cristian Varela
Max Chapman
The Strumbellas
The Lone Bellow
Ambrosia
Yngwie J. Malmsteen
DILANA
Like Mike
The Time
Teebs
Capsize
Graveyard
Brett Newski
Grizzly
McCoy Tyner
Itzhak Perlman
Bill Maher
MSP
Devendra Banhart
Klangkarussell
Blaggards
New Politics
BulletBoys
Smoke
Armand van Helden
Diana R

Aoife O'Donovan
Dokken
Mickey
Garbage
Fritz Kalkbrenner
Fred P
Vibesquad
Tera Melos
Motor City Drum Ensemble
Terrace
Bruce Hornsby
French Horn Rebellion
Corey Harris
Prairie Home Companion
Brendan James
Ólafur Arnalds
Nazareth
Fat Joe
Fiji
Violin
Walker & Royce
Drake White
Tyrone Wells
Manic Street Preachers
Monophonics
Lauren Flax
Break Science
Jason Eady
TroyBoi
Jess Klein
Ghostland Observatory
The Browning
the Nightcats
Remedy Drive
Birds of Chicago
Oscar Mulero
Joshua James
The Ukulele Orchestra Of Great Britain
Monoloc
Jason Gray
Neal McCoy
The Field
Bobby Long
Futuristic Polar Bears
Willy Porter
Wye Oak
Moon Hooch
Bands
WeareTreo
FIDLAR
DJ Qu
St. Paul and the Broken Bones
Angel Olsen
THE MAIN SQUEEZE
Cory Chisel
Margot
Trentemøller
Hamlet
Iced Earth
Sleepy Sun
EDEN 
Tantrum Desire
Tony Lucca
The Sleeping Souls
Omar Souleyman
Akira
emalkay
Kyle Park
Porter
John Waters
Chic
Amos Lee
The Shins
Elliott BROOD
Otep
Oscar G.
Vince Staples
Soulwax
Marcellus Pittman
The Bodyguard
Britt Ni

Terrence Parker
Doug MacLeod
Nickelback
Evidence
Josh Abbott Band
Jon Pardi
VADER
Jim Jeffries
Conor Oberst
2Cellos
Mø
Ben Wendel
Burn
Status Quo
Warrant
Lukas
Human Nature
Heatwave
Le Vent du Nord
Pet Shop Boys
Nightmares on Wax
Tim O'Brien
Mushroomhead
MYKKI BLANCO
Actress
Enslaved
Ludacris
Joan as Police Woman
Local Showcase
Nekromantix
Strings
Sophie
Seven
Cash'd Out
Lianne La Havas
Peppino D'Agostino
Bella Sarris
State Champs
Pa
OK Go
Papadosio
The Black Lillies
Echo & The Bunnymen
Truncate
Empire of the Sun
Tedeschi Trucks Band
Tribal Seeds
Big Head Todd and The Monsters
Beth Orton
High Voltage
Dreamer
The Departed
Kyle Hall
Honey Dijon
Morten
Canned Heat
Peter Murphy
Charlie Wilson
Mobb Deep
Flash
Kid Rock
Stone
The Maccabees
Mad Caddies
Barcelona
Paul Rodgers
Saxon
Eelke Kleijn
Young the Giant
Sweater Beats
USA
John Gorka
Alex Niggemann
Ma
Miyagi
Ben Sims
Peter Frampton
Jon Hopkins
M.I.A.
Adelaide
65daysofstatic
Avatar
GWAR
Ahmad Jamal
Lizz Wright
BETRAYING THE MARTYRS
DIRTY AU

Baroness
Common
The New Pornographers
Imagine Dragons
The Real Pete Rock
X Ambassadors
Stanley Clarke
Gene Farris
Mandisa
Sheryl Crow
Jim
David Puentez
Chris Ayer
Kate Nash
Liquid Stranger
Chaim
Liars
Casting Crowns
DMX
Katatonia
Country
Devin Townsend
Lewis Black
One Night of Queen
GBH
Wu-Tang Clan
ProtoHype
Chevelle
Mindi Abair
Jeremy Olander
Wynonna
Kirk Whalum
The Creepshow
Detroit Swindle
Odesza
Arek King
Kristin Kontrol
Roxie's Rock n Roll Parking Lot
Bonfire
Warm Up
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
Neil Giraldo
Kill Paris
Fury
Murs
Everclear
Le Youth
The Stylistics
Toby Keith
Weezer
Bowling for Soup
Placebo
Mel DeBarge
Bro Safari
Night Ranger
Tom Trago
Alix Perez
YokoO
Tab Benoit
Iration
Supernova (IT)
Phil Wickham
Pink
Betty
Sepultura
Hybrid
Suicidal Tendencies
Matt
Eric
Eliot Lipp
Marco Faraone
Robyn Hitchcock
Joe Satriani
Goo

The Game
Black & White
Scott H. Biram
Coyu
Safari
Fat Freddy's Drop
Real Estate
Fort Knox Five
Greensky Bluegrass
Shooter Jennings
Patrick Topping
Sweet
Willy Monfret
The Ten Tenors
Tom Jones
Collie Buddz
Anna Lunoe
Touche Amore
Animal Collective
James Blake
Eric Burdon
The Jillionaire
Wiwek
EOTO
Wine
Norma Jean
Indigo Girls
Man
Randy Houser
Bicep
Scorpions
LCD Soundsystem
Pat Benatar
Nicole Moudaber
Outlaws
Boz Scaggs
The B-52's
Jesse Malin
Bellamy Brothers Band
NEEDTOBREATHE
Sebastian Ingrosso
Clark
The Diamonds
GLOWINTHEDARK
Bob Wayne
Thee Oh Sees
Concha Buika
Tinariwen
James Blunt
Techno
The Vaccines
Droog
Benji
Ryan Adams
Interpol
Gov't Mule
Tift Merritt
Dustin Lynch
Body
Sonata Árctica
Death Angel
Randy Rogers Band
Mando Diao
Brooke Evers
Nosaj Thing
Josh Turner
Paul Anka
Steve Poltz
Gladys Knight
Third Eye Blind
King Parrot
Anais Mitchell
Diana Krall
Action Bronson
Stray From the Path
Brother Ali
Omar-S
Supports
Lee Brice
Sick Puppies
Kenny Larkin
Blood, Sweat & Tears
Sidewalk P

DJ Fresh
The Rat Pack
Horse Meat Disco
The Brew
Ben
Fashen
Silversun Pickups
Macy Gray
Gin Blossoms
Lionel Loueke
Pretty Lights
Black Stone Cherry
Tech N9ne
Epica
Galactic
Chali 2na
Yellow Claw
Rick Springfield
Closed For Private Event
Born of Osiris
Camea
Gretchen Peters
doctor P
MercyMe
Gregory Porter
Keith Sweat
Kataklysm
Downlink
Tesla
Dropkick Murphys
Beach House
Building 429
Wolfgang Gartner
Jonny Lang
Wilkinson
Sven Väth
Simple Plan
Hawk Nelson
Travis Tritt
The Flaming Lips
The Soul Rebels
Supersuckers
b2b
Sigma
Martin
Seether
Dierks Bentley
André Rieu
Dionne Warwick
Art
Béla Fleck
Little Big Town
Madeon
Phoenix
Jackmaster
Killswitch Engage
David Morales
Ellis Paul
!!!
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3

Marcel Dettmann
DJ Snake
BLOND:ISH
The Juan MacLean
Carmen
Less Than Jake
Matthew Dear
Andy C
Tiefschwarz
Stick to Your Guns
Joe Maz
Ruthie Foster
Heidi
DJ Diamond Fan Page
Yves V
Satoshi Tomiie
Chus+Ceballos
Matador
Menno de Jong
ATB
Talib Kweli
August Burns Red
Dannic
Richie Hawtin
Deadmau5
Closed
AC Slater
Jerome Isma-Ae
Host
White
Anja Schneider
KO:YU
SICK INDIVIDUALS
Crystal Castles
Suicide Silence
Joe Bonamassa
You
Justin Martin
Sold Out
The Upbeats
Carlo Lio
Boyz II Men
Kings of Leon
Dirtyphonics
A Day To Remember
Robert Glasper
Rufus Wainwright
Chris Liebing
Air Supply
PRIVATE EVENT
Showtek
Terror
DJ Politik
Morgan Page
Parkway Drive
Steel Panther
The Martinez Brothers
Dillon Francis
Joachim Garraud
Cancelled
Kryoman
Jesse Marco
Tiga
Derrick May
Alex
Nick Warren
Tommy Emmanuel
Butch
Chris Lake
Beethoven
Bryan Adams
Battle of the Bands
Andrew Rayel
Bassjackers
M.A.N.D.Y.
[unknown]
DJ Carnage
Viper Recordings
Los Lobos
Alesso
Dave
Skream
The Beach Boys
Stacey Pullen
Timo Maas
Zed

In [ ]:
     with open("./my_nodesdictGLOBAL.obj", 'rb') as handle:
          my_nodesdictGLOBAL= pickle.load( handle)

In [31]:
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

print take(5,my_nodesdictGLOBAL.iteritems())

[('pete-s-place32-117', {'data': {'count': 1, 'end': '2017-04-01T19:00:00', 'name': 'pete-s-place', 'weight': 1.0, 'color': '#7f523f', 'notes': '  \n List of the bands that played in this venue:  \nblack-joe-lewis  \n TOTAL: 1 Bands', 'start': '2017-03-31T19:00:00', 'lat': 32.7677778, 'lng': -117.0222222, 'id': 'pete-s-place32-117'}}), ('tucson-music-hall32-110', {'data': {'count': 35, 'end': '2018-01-15T14:00:00', 'name': 'tucson-music-hall', 'weight': 7.158396516477644, 'color': '#3f7f6c', 'notes': '  \n List of the bands that played in this venue:  \ncherish-the-ladies  \nitzhak-perlman  \nbill-maher  \ncurtis-stigers  \njonathan-richman  \nthe-lettermen  \npiotr-ilich-chaikovskii  \nthe-midtown-men  \njoseph-haydn  \ngeorge-lopez  \nsuper-diamond  \njane-s-addiction  \nlindsey-stirling  \narturo-sandoval  \nlewis-black  \ndon-giovanni  \njerry-seinfeld  \nkt-tunstall  \nvideo-games-live  \nthe-ten-tenors  \nbellamy-brothers-band  \nbrian-regan  \nmichael-feinstein  \nrain  \njackso

In [22]:
# credentials
TOPOGRAM_URL = "http://localhost:3000"
USER = "gregory.bahde@laposte.net"
PASSWORD = "matrix"

In [33]:
print len(my_nodesdictGLOBAL)

127691


In [23]:
# connect to the topogram instance (pass debug=True params for more info )
topogram = TopogramAPIClient(TOPOGRAM_URL) #, debug=True)

# create a new user
try :
    topogram.create_user(USER, PASSWORD)
except ValueError:
    print "> User has already been created."

# login a new user if needed
resp_user_login = topogram.user_login(USER, PASSWORD)
print resp_user_login

assert(resp_user_login["status"] == "success")
assert(resp_user_login["status_code"] == 200)


ERROR:topogram-client:403 Unauthorized request


> User has already been created.
{u'status': u'success', 'status_code': 200, u'data': {u'authToken': u'axDpwL51fT-J6E2wNggDXKEOYSRq4_0MgMaf3R9FWgY', u'userId': u'BxRn4kp4WFL92jW8g'}}


In [24]:
###IMPORT ALL THE BANDS GRAPHS

nodestop=[]
edgestop=[]
tit=[]
import glob, os

print os.getcwd()

for subdir, dirs, files in os.walk(savpath): 
 for file in files:
  if file.endswith('.obj') == True:  
    with open(os.path.join(subdir, file), 'rb') as handle:
          temp= pickle.load( handle)
    #print 
    tit=temp[0].encode("utf-8")
    edgestop=temp[2]
    nodestop=temp[1]
    #print edgestop
    #for edge in edgestop:
    #    assert edge["data
    while True:
        try:
            #print
            create_topogram(tit, nodestop, edgestop)
        except TypeError:
            continue
        break
print 'DONE'
        

ERROR:topogram-client:500 - Error : A topogram with the same name already exists


/home/goonieb/gitrep/bandstour/5_visualizer/topogram-api-clientFOR_TOPO_FROM_BANDSTOUR
Creating topogram 'The Real McKenzies/BETA_0.3'
{u'status': u'success', 'status_code': 200, u'data': u'QpKepMm2T6ngwdbgB', u'statusCode': 201}
Creating topogram 'The Real McKenzies/BETA_0.3'
> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u'The Real McKenzies/BETA_0.3', u'userId': u'BxRn4kp4WFL92jW8g', u'_id': u'QpKepMm2T6ngwdbgB', u'slug': u'the-real-mckenziesbeta_03', u'createdAt': u'2018-05-19T21:42:06.994Z'}, u'statusCode': 200}
topogram ID : QpKepMm2T6ngwdbgB
274 nodes created.
322 edges created.
done. Topogram has been updated. Check it at http://localhost:3000/topograms/QpKepMm2T6ngwdbgB
Creating topogram 'D.O.D/BETA_0.3'


ERROR:topogram-client:500 - Error : A topogram with the same name already exists


{u'status': u'success', 'status_code': 200, u'data': u'F9TfEGnfAhpWj2XAS', u'statusCode': 201}
Creating topogram 'D.O.D/BETA_0.3'
> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u'D.O.D/BETA_0.3', u'userId': u'BxRn4kp4WFL92jW8g', u'_id': u'F9TfEGnfAhpWj2XAS', u'slug': u'dodbeta_03', u'createdAt': u'2018-05-19T21:42:08.211Z'}, u'statusCode': 200}
topogram ID : F9TfEGnfAhpWj2XAS


ERROR:topogram-client:500 - Error : A topogram with the same name already exists


...

> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u'Speakeasy/BETA_0.3', u'userId': u'BxRn4kp4WFL92jW8g', u'_id': u'QQdzFrGvYK3AtLDFm', u'slug': u'speakeasybeta_03', u'createdAt': u'2018-05-19T23:07:04.476Z'}, u'statusCode': 200}
topogram ID : QQdzFrGvYK3AtLDFm


ERROR:topogram-client:500 - Error : A topogram with the same name already exists


22 nodes created.
38 edges created.
done. Topogram has been updated. Check it at http://localhost:3000/topograms/QQdzFrGvYK3AtLDFm
Creating topogram 'The Front Bottoms/BETA_0.3'
{u'status': u'success', 'status_code': 200, u'data': u'XDm6x4T2dN2knLi69', u'statusCode': 201}
Creating topogram 'The Front Bottoms/BETA_0.3'
> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u'The Front Bottoms/BETA_0.3', u'userId': u'BxRn4kp4WFL92jW8g', u'_id': u'XDm6x4T2dN2knLi69', u'slug': u'the-front-bottomsbeta_03', u'createdAt': u'2018-05-19T23:07:05.792Z'}, u'statusCode': 200}
topogram ID : XDm6x4T2dN2knLi69
418 nodes created.


ERROR:topogram-client:500 - Error : A topogram with the same name already exists


499 edges created.
done. Topogram has been updated. Check it at http://localhost:3000/topograms/XDm6x4T2dN2knLi69
Creating topogram 'Ill-Esha/BETA_0.3'
{u'status': u'success', 'status_code': 200, u'data': u'5oP8Mbd5jCFTwWQQh', u'statusCode': 201}
Creating topogram 'Ill-Esha/BETA_0.3'
> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u'Ill-Esha/BETA_0.3', u'userId': u'BxRn4kp4WFL92jW8g', u'_id': u'5oP8Mbd5jCFTwWQQh', u'slug': u'ill-eshabeta_03', u'createdAt': u'2018-05-19T23:07:08.421Z'}, u'statusCode': 200}
topogram ID : 5oP8Mbd5jCFTwWQQh
231 nodes created.
273 edges created.
done. Topogram has been updated. Check it at http://localhost:3000/topograms/5oP8Mbd5jCFTwWQQh


In [50]:
###IMPORT THE GLOBAL VENUE  GRAPH
my_nodesdictList=[]
nodestopo=[]
edgestopo=[]
with open("./my_nodesdictGLOBAL.obj", 'rb') as handle:
          my_nodesdictGLOBAL= pickle.load( handle)
        
for key, value in my_nodesdictGLOBAL.iteritems():
            my_nodesdictGLOBAL[key]["data"].pop('count')
        
for key, value in my_nodesdictGLOBAL.iteritems():
            my_nodesdictList.append(value)
        
nodestopo=sorted(my_nodesdictList, key= lambda (v) : v['data']['weight'], reverse=True)[:5000]
print len(nodestopo)



title="global venue graph /BETA 0.3"

while True:
        try:
            print nodestopo
            create_topogram(title, nodestopo, edgestopo)
        
        except TypeError:
            continue
        break
        

ERROR:topogram-client:500 - Error : A topogram with the same name already exists
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


5000 nodes deleted
5000 nodes created.


ValueError: No JSON object could be decoded